In [1]:
from itertools import permutations

import pandas as pd

from glicko_rating_matches import GlickoSystem, Team
from lol_fandom import get_tournaments, get_match_schedule

pd.set_option("display.max_columns", None)

In [2]:
def get_team_id(teams_id, team_name):
    return teams_id.loc[teams_id["team"] == team_name, "team_id"].iloc[0]

In [18]:
teams_id = pd.read_csv("./csv/teams_id.csv")
ratings = pd.read_csv(
    "./csv/glicko_rating/glicko_rating_2023.csv", parse_dates=["last_game_date"]
)
scoreboard_games = pd.read_csv("./csv/scoreboard_games/2023_scoreboard_games.csv")
teams_id.shape, ratings.shape, scoreboard_games.shape

((757, 2), (598, 15), (2179, 37))

In [21]:
scoreboard_games.loc[scoreboard_games['League'] == 'LCK'].tail(6)

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision,League
2104,LCK/2023 Season/Summer Season,Nongshim RedForce,Dplus KIA,Dplus KIA,Nongshim RedForce,2023-06-18 09:30:00,0,2,2,31:18,31.300000,"Maokai,Azir,Draven,Viego,Renekton","Neeko,Vi,Lucian,Annie,Jax","Gwen,Sejuani,Cassiopeia,Jinx,Milio","K'Sante,Lee Sin,LeBlanc,Aphelios,Yuumi","DnDn,Sylvie,Quad,Jiwoo,Peter (Jeong Yoon-su)","Canna,Canyon,ShowMaker,Deft,Kellin",0.0,4.0,0.0,1.0,1.0,11.0,51082.0,60246.0,9.0,12.0,2.0,0.0,0.0,3.0,13.11,LCK/2023 Season/Summer Season_Week 2_10_2,LCK/2023 Season/Summer Season_Week 2_10,NaN,0,LCK
2150,LCK/2023 Season/Summer Season,Liiv SANDBOX,OKSavingsBank BRION,Liiv SANDBOX,OKSavingsBank BRION,2023-06-21 08:06:00,1,0,1,34:29,34.483333,"Renekton,Annie,Yuumi,Zeri,Varus","Milio,Aphelios,Nautilus,Viego,Sejuani","Gnar,Wukong,Neeko,Xayah,Rakan","K'Sante,Vi,LeBlanc,Jinx,Tahm Kench","Burdol,Willer,Clozer,Teddy,Kael (Kim Jin-hong)","Morgan,UmTi,Ivory,Hena,Effort",5.0,0.0,2.0,0.0,11.0,4.0,68955.0,58438.0,15.0,8.0,1.0,1.0,3.0,0.0,13.11,LCK/2023 Season/Summer Season_Week 3_1_1,LCK/2023 Season/Summer Season_Week 3_1,NaN,0,LCK
2152,LCK/2023 Season/Summer Season,Liiv SANDBOX,OKSavingsBank BRION,OKSavingsBank BRION,Liiv SANDBOX,2023-06-21 09:02:00,1,1,2,40:37,40.616667,"Renekton,Yuumi,Neeko,Annie,Maokai","Milio,Aphelios,Vi,Varus,Jinx","K'Sante,Sejuani,LeBlanc,Zeri,Lulu","Gwen,Poppy,Tristana,Xayah,Rakan","Burdol,Willer,Clozer,Teddy,Kael (Kim Jin-hong)","Morgan,UmTi,Ivory,Hena,Effort",2.0,4.0,1.0,2.0,7.0,10.0,73758.0,80480.0,7.0,16.0,1.0,1.0,1.0,1.0,13.11,LCK/2023 Season/Summer Season_Week 3_1_2,LCK/2023 Season/Summer Season_Week 3_1,NaN,0,LCK
2154,LCK/2023 Season/Summer Season,Liiv SANDBOX,OKSavingsBank BRION,Liiv SANDBOX,OKSavingsBank BRION,2023-06-21 10:04:00,2,1,1,31:07,31.116667,"Renekton,Yuumi,Poppy,Rakan,Viego","Milio,Aphelios,Vi,Varus,Zeri","Jax,Sejuani,LeBlanc,Lucian,Nami","Gragas,Neeko,Tristana,Xayah,Nautilus","Burdol,Willer,Clozer,Teddy,Kael (Kim Jin-hong)","Morgan,UmTi,Ivory,Hena,Effort",2.0,2.0,1.0,0.0,9.0,2.0,61912.0,52567.0,13.0,6.0,2.0,0.0,1.0,0.0,13.11,LCK/2023 Season/Summer Season_Week 3_1_3,LCK/2023 Season/Summer Season_Week 3_1,NaN,0,LCK
2157,LCK/2023 Season/Summer Season,Kwangdong Freecs,T1,T1,Kwangdong Freecs,2023-06-21 11:42:00,0,1,2,30:60,31.000000,"Kindred,Vi,Annie,Wukong,Viego","Neeko,LeBlanc,K'Sante,Taliyah,Yone","Renekton,Sejuani,Ezreal,Zeri,Yuumi","Gragas,Poppy,Azir,Aphelios,Milio","DuDu (Lee Dong-ju),YoungJae,BuLLDoG (Lee Tae-y...","Zeus,Oner,Faker,Gumayusi,Keria",0.0,4.0,1.0,0.0,3.0,9.0,53162.0,60160.0,4.0,18.0,2.0,0.0,0.0,1.0,13.11,LCK/2023 Season/Summer Season_Week 3_2_1,LCK/2023 Season/Summer Season_Week 3_2,NaN,0,LCK
2159,LCK/2023 Season/Summer Season,Kwangdong Freecs,T1,T1,Kwangdong Freecs,2023-06-21 12:34:00,0,2,2,27:32,27.533333,"Kindred,Vi,Annie,Wukong,Renekton","Neeko,LeBlanc,K'Sante,Yone,Taliyah","Jax,Sejuani,Tristana,Zeri,Yuumi","Gragas,Poppy,Azir,Aphelios,Milio","DuDu (Lee Dong-ju),YoungJae,BuLLDoG (Lee Tae-y...","Zeus,Oner,Faker,Gumayusi,Keria",0.0,3.0,0.0,1.0,1.0,11.0,43010.0,57092.0,5.0,19.0,2.0,0.0,0.0,3.0,13.11,LCK/2023 Season/Summer Season_Week 3_2_2,LCK/2023 Season/Summer Season_Week 3_2,NaN,0,LCK
2165,LCK/2023 Season/Summer Season,Gen.G,DRX,Gen.G,DRX,2023-06-22 08:07:00,1,0,1,22:53,22.883333,"Sejuani,Maokai,Vi,Poppy,Lee Sin","LeBlanc,Neeko,Annie,Wukong,Nocturne","Renekton,Kha'Zix,Azir,Aphelios,Yuumi","Jax,Viego,Ahri,Draven,Milio","Doran (Choi Hyeon-joon),Peanut,Chovy,Peyz,Delight","Rascal,Juhan,FATE (Yoo Su-hyeok),Paduck,BeryL",3.0,0.0,1.0,0.0,8.0,1.0,48134.0,34117.0,11.0,0.0,1.0,1.0,1.0,0.0,13.11,LCK/2023 Season/Summer Season_Week 3_3_1,LCK/2023 Season

In [4]:
ratings.loc[(ratings["league"] == "LCK") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
1,220,Gen.G,LCK,1024,614,410,0.599609,7,2025.678016,46.686849,0.565101,432.483068,229.308044,2023-06-22 08:52:00,- (+2)
2,69,T1,LCK,1446,960,486,0.663900,6,2006.859234,56.087865,0.566006,571.053414,301.503586,2023-06-21 12:34:00,- (-)
4,219,KT Rolster,LCK,1057,628,429,0.594134,6,1959.257549,70.114352,0.570862,438.657278,232.655541,2023-06-22 11:36:00,- (+21)
7,508,Dplus KIA,LCK,629,440,189,0.699523,-2,1844.971306,69.935458,0.566630,233.396783,122.167622,2023-06-22 11:36:00,-1 (-21)
9,205,Hanwha Life Esports,LCK,863,429,434,0.497103,4,1838.648796,69.823991,0.576615,366.674118,193.211938,2023-06-17 07:04:00,- (-)
27,509,Liiv SANDBOX,LCK,456,217,239,0.475877,1,1731.309680,69.296357,0.572028,200.945191,105.576062,2023-06-21 10:04:00,-1 (-)
35,301,Kwangdong Freecs,LCK,911,430,481,0.472009,-6,1706.823392,56.246637,0.574046,406.058926,216.749628,2023-06-21 12:34:00,- (-)
79,33,DRX,LCK,1109,572,537,0.515780,-7,1628.813367,46.734994,0.569029,442.378369,226.915644,2023-06-22 08:52:00,-2 (-2)
95,15,OKSavingsBank BRION,LCK,500,180,320,0.360000,-1,1603.193365,69.060699,0.564360,206.632340,110.912937,2023-06-21 10:04:00,- (-)
119,524,Nongshim RedForce,LCK,306,116,190,0.379085,-6,1573.088248,56.860876,0.575046,129.967686,69.890944,2023-06-18 09:30:00,-1 (-)


In [23]:
scoreboard_games.loc[scoreboard_games['League'] == 'LPL'].tail(9)

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision,League
2156,LPL/2023 Season/Summer Season,Bilibili Gaming,LNG Esports,Bilibili Gaming,LNG Esports,2023-06-21 11:15:00,1,0,1,28:42,28.700000,"Jayce,Milio,Yuumi,Poppy,Gwen","Vi,Kindred,Neeko,Xayah,Zeri","Jax,Wukong,Annie,Jinx,Rakan","Renekton,Maokai,Azir,Aphelios,Nautilus","Bin (Chen Ze-Bin),Xun,Yagao,Elk,ON","Zika,Tarzan (Lee Seung-yong),Scout,GALA,Hang",3.0,1.0,1.0,0.0,9.0,3.0,58472.0,48263.0,19.0,9.0,NaN,NaN,1.0,0.0,13.11,LPL/2023 Season/Summer Season_Week 4_7_1,LPL/2023 Season/Summer Season_Week 4_7,NaN,0,LPL
2158,LPL/2023 Season/Summer Season,LNG Esports,Bilibili Gaming,Bilibili Gaming,LNG Esports,2023-06-21 12:13:00,0,2,2,33:03,33.050000,"Vi,Rakan,Yuumi,Kha'Zix,Kindred","Annie,Neeko,Jayce,Poppy,Sejuani","Ornn,Wukong,LeBlanc,Tristana,Milio","Renekton,Nidalee,Syndra,Aphelios,Blitzcrank","Zika,Tarzan (Lee Seung-yong),Scout,GALA,Hang","Bin (Chen Ze-Bin),Xun,Yagao,Elk,ON",2.0,3.0,0.0,2.0,2.0,10.0,57214.0,67777.0,15.0,21.0,NaN,NaN,0.0,2.0,13.11,LPL/2023 Season/Summer Season_Week 4_7_2,LPL/2023 Season/Summer Season_Week 4_7,NaN,0,LPL
2164,LPL/2023 Season/Summer Season,Invictus Gaming,Rare Atom,Rare Atom,Invictus Gaming,2023-06-22 07:15:00,0,1,2,28:03,28.050000,"Renekton,Yuumi,K'Sante,Varus,Xayah","Neeko,Milio,Annie,Lucian,Draven","Fiora,Vi,Ahri,Aphelios,Rakan","Jax,Maokai,Jayce,Jinx,Nautilus","YSKM,Tianzhen,Cryin,Ahn (An Shan-Ye),Wink","Xiaoxu,Leyan,Strive,Assum,Baolan",2.0,2.0,0.0,1.0,3.0,11.0,45869.0,55531.0,5.0,15.0,NaN,NaN,0.0,2.0,13.11,LPL/2023 Season/Summer Season_Week 4_8_1,LPL/2023 Season/Summer Season_Week 4_8,NaN,0,LPL
2166,LPL/2023 Season/Summer Season,Rare Atom,Invictus Gaming,Rare Atom,Invictus Gaming,2023-06-22 08:07:00,2,0,1,36:46,36.766667,"Milio,Maokai,Annie,Lissandra,LeBlanc","Renekton,Nidalee,Neeko,Nautilus,Rakan","Jax,Vi,Ahri,Aphelios,Lulu","Kennen,Rengar,Tristana,Zeri,Yuumi","Xiaoxu,Leyan,Strive,Assum,Baolan","YSKM,Tianzhen,Cryin,Ahn (An Shan-Ye),Wink",3.0,3.0,3.0,0.0,10.0,3.0,73265.0,65407.0,17.0,17.0,NaN,NaN,1.0,0.0,13.11,LPL/2023 Season/Summer Season_Week 4_8_2,LPL/2023 Season/Summer Season_Week 4_8,NaN,0,LPL
2168,LPL/2023 Season/Summer Season,Royal Never Give Up,EDward Gaming,Royal Never Give Up,EDward Gaming,2023-06-22 09:35:00,1,0,1,27:19,27.316667,"Lucian,Milio,Yuumi,Rakan,Renekton","Vi,Neeko,Poppy,Lulu,Thresh","K'Sante,Wukong,Annie,Aphelios,Braum","Gwen,Maokai,Jayce,Xayah,Nautilus","Breathe,Wei (Yan Yang-Wei),Tangyuan,LP,LvMao","Ale,JieJie,FoFo,Uzi (Jian Zi-Hao),Meiko",3.0,1.0,1.0,0.0,9.0,3.0,54799.0,45327.0,12.0,3.0,NaN,NaN,2.0,0.0,13.11,LPL/2023 Season/Summer Season_Week 4_9_1,LPL/2023 Season/Summer Season_Week 4_9,NaN,0,LPL
2169,LPL/2023 Season/Summer Season,EDward Gaming,Royal Never Give Up,Royal Never Give Up,EDward Gaming,2023-06-22 10:24:00,0,2,2,28:39,28.650000,"Neeko,Vi,Annie,Sion,Ahri","Lucian,Milio,Aphelios,K'Sante,Renekton","Gragas,Poppy,Azir,Zeri,Yuumi","Ornn,Kha'Zix,LeBlanc,Draven,Nautilus","Ale,JieJie,FoFo,Uzi (Jian Zi-Hao),Meiko","Breathe,Wei (Yan Yang-Wei),Tangyuan,LP,LvMao",0.0,4.0,0.0,1.0,1.0,10.0,46008.0,56148.0,7.0,10.0,NaN,NaN,0.0,2.0,13.11,LPL/2023 Season/Summer Season_Week 4_9_2,LPL/2023 Season/Summer Season_Week 4_9,NaN,0,LPL
2171,LPL/2023 Season/Summer Season,Team WE,Oh My God,Team WE,Oh My God,2023-06-22 11:16:00,1,0,1,27:12,27.200000,"Neeko,Yuumi,Milio,Ahri,LeBlanc","Jinx,Lucian,Renekton,Lulu,Rakan","K'Sante,Maokai,Jayce,Zeri,Braum","Darius,Vi,Annie,Tristana,Nautilus","Cube (Dai Yi),Heng,Shanks,Hope (Wang Jie),Iwandy","shanji,Aki (Mao An),Creme,Able,ppgod",2.0,2.0,0.0,0.0,7.0,2.0,51403.0,45222.0,16.0,5.0,NaN,NaN,1.0,0.0,13.11,LPL/2023 Season/Summer Season_Week 4

In [24]:
ratings.loc[(ratings["league"] == "LPL") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
0,444,JD Gaming,LPL,742,440,302,0.592992,4,2047.693775,69.358026,0.540121,303.437995,155.588232,2023-06-18 12:13:00,- (-)
3,458,Bilibili Gaming,LPL,556,291,265,0.523381,2,1972.532740,67.104517,0.539238,251.695430,131.895096,2023-06-21 12:13:00,- (-)
13,244,LNG Esports,LPL,842,423,419,0.502375,-2,1815.361153,67.491745,0.552233,387.907519,206.445421,2023-06-21 12:13:00,- (-)
14,461,Top Esports,LPL,698,404,294,0.578797,1,1811.632620,70.257296,0.547176,282.573745,143.984136,2023-06-18 10:49:00,- (-)
16,60,Oh My God,LPL,1047,484,563,0.462273,2,1788.242023,70.500678,0.555717,443.555313,227.137402,2023-06-22 12:59:00,+2 (+7)
17,445,Weibo Gaming,LPL,574,314,260,0.547038,3,1785.684426,61.368995,0.556434,260.656314,139.453966,2023-06-21 10:31:00,- (-)
19,305,Royal Never Give Up,LPL,1161,749,412,0.645134,2,1758.213326,71.140493,0.555348,480.758466,254.212617,2023-06-22 10:24:00,+9 (+32)
20,170,EDward Gaming,LPL,1210,744,466,0.614876,-2,1757.983786,70.865984,0.550910,503.838554,265.332158,2023-06-22 10:24:00,-4 (-31)
43,39,Team WE,LPL,1118,587,531,0.525045,-2,1692.257048,71.057435,0.573737,502.730782,263.715757,2023-06-22 12:59:00,-3 (-6)
76,515,Ninjas in Pyjamas.CN,LPL,385,165,220,0.428571,-2,1632.363488,71.043348,0.549363,168.119152,90.955498,2023-06-19 10:15:00,- (-)


In [25]:
scoreboard_games.loc[scoreboard_games['League'] == 'LEC'].tail()

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision,League
2130,LEC/2023 Season/Summer Season,Excel Esports,SK Gaming,SK Gaming,Excel Esports,2023-06-19 16:08:00,0,1,2,37:21,37.350000,"Vi,LeBlanc,Maokai,Malphite,Ahri","Neeko,Zeri,Draven,Poppy,Kha'Zix","K'Sante,Viego,Annie,Aphelios,Yuumi","Gnar,Wukong,Lissandra,Jinx,Milio","Odoamne,Peach,Abbedagge,Patrik,LIMIT (Dino Tot)","Irrelevant,Markoon,Sertuss,Exakick,Doss",1.0,4.0,2.0,0.0,8.0,7.0,67151.0,64509.0,10.0,15.0,1.0,1.0,2.0,2.0,13.11,LEC/2023 Season/Summer Season_Week 1_11_1,LEC/2023 Season/Summer Season_Week 1_11,NaN,0,LEC
2131,LEC/2023 Season/Summer Season,Team BDS,Astralis,Astralis,Team BDS,2023-06-19 17:11:00,0,1,2,25:38,25.633333,"Irelia,Lucian,Maokai,Rakan,Kennen","Vi,LeBlanc,Zeri,Poppy,Wukong","Renekton,Gragas,Neeko,Aphelios,Milio","Rumble,Sejuani,Yasuo,Xayah,Blitzcrank","Adam (Adam Maanane),Sheo,nuc,Crownie,Labrov","Finn,113,LIDER,Kobbe,JeongHoon",0.0,4.0,0.0,1.0,1.0,11.0,39170.0,51926.0,3.0,15.0,0.0,2.0,0.0,3.0,13.11,LEC/2023 Season/Summer Season_Week 1_12_1,LEC/2023 Season/Summer Season_Week 1_12,NaN,0,LEC
2132,LEC/2023 Season/Summer Season,MAD Lions,KOI (Spanish Team),MAD Lions,KOI (Spanish Team),2023-06-19 18:08:00,1,0,1,40:19,40.316667,"Zeri,LeBlanc,Neeko,Nautilus,Jarvan IV","Vi,Yuumi,Milio,Jayce,Renekton","Gragas,Wukong,Annie,Aphelios,Rakan","K'Sante,Nocturne,Ahri,Xayah,Leona","Chasy,Elyoya,Nisqy,Carzzy,Hylissang","Szygenda,Malrang,Larssen,Comp,Advienne",3.0,2.0,1.0,1.0,10.0,5.0,72641.0,70036.0,16.0,18.0,0.0,2.0,3.0,0.0,13.11,LEC/2023 Season/Summer Season_Week 1_13_1,LEC/2023 Season/Summer Season_Week 1_13,NaN,0,LEC
2133,LEC/2023 Season/Summer Season,G2 Esports,Fnatic,G2 Esports,Fnatic,2023-06-19 19:18:00,1,0,1,31:26,31.433333,"Vi,Yuumi,Blitzcrank,Kindred,Nocturne","Draven,LeBlanc,Kalista,Jayce,Syndra","K'Sante,Maokai,Ahri,Aphelios,Milio","Malphite,Viego,Neeko,Lucian,Nami","BrokenBlade,Yike,Caps,Hans Sama,Mikyx","Oscarinin,Razork,Humanoid,Noah (Oh Hyeon-taek)...",4.0,1.0,0.0,1.0,9.0,2.0,60176.0,50776.0,25.0,8.0,2.0,0.0,1.0,0.0,13.11,LEC/2023 Season/Summer Season_Week 1_14_1,LEC/2023 Season/Summer Season_Week 1_14,NaN,0,LEC
2134,LEC/2023 Season/Summer Season,Team Heretics,Team Vitality,Team Heretics,Team Vitality,2023-06-19 20:17:00,1,0,1,33:03,33.050000,"Xayah,Neeko,Yuumi,Rakan,Lulu","Draven,Vi,Lucian,K'Sante,Renekton","Sion,Maokai,Kai'Sa,Zeri,Milio","Gwen,Wukong,LeBlanc,Aphelios,Nautilus","Evi,Jankos,Vetheo,Flakked,Mersa","Photon,Bo,Perkz,Upset,Kaiser",3.0,2.0,0.0,2.0,6.0,6.0,58937.0,60286.0,11.0,8.0,1.0,1.0,1.0,0.0,13.11,LEC/2023 Season/Summer Season_Week 1_15_1,LEC/2023 Season/Summer Season_Week 1_15,NaN,0,LEC


In [26]:
ratings.loc[(ratings["league"] == "LEC") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
6,211,G2 Esports,LEC,1016,580,436,0.570866,1,1863.036428,64.826826,0.594672,400.866629,210.259661,2023-06-19 19:18:00,+1 (-)
23,333,MAD Lions,LEC,679,337,342,0.496318,3,1750.177284,68.921153,0.594889,303.350004,161.646514,2023-06-19 18:08:00,-1 (-)
59,598,Team BDS,LEC,75,31,44,0.413333,-1,1658.831065,69.614370,0.595004,31.631608,17.241279,2023-06-19 17:11:00,- (-)
72,3,Fnatic,LEC,1246,760,486,0.609952,-1,1636.730683,64.870817,0.594767,513.555152,271.987046,2023-06-19 19:18:00,- (-)
91,334,Team Vitality,LEC,389,173,216,0.444730,-1,1606.645729,72.005565,0.595121,174.184021,92.230202,2023-06-19 20:17:00,- (-)
117,611,KOI (Spanish Team),LEC,41,18,23,0.439024,-2,1580.376960,69.140431,0.595001,19.283699,10.646059,2023-06-19 18:08:00,- (-)
145,612,Team Heretics,LEC,27,11,16,0.407407,3,1547.988788,72.119542,0.595193,12.895574,7.008752,2023-06-19 20:17:00,- (-)
146,310,Astralis,LEC,460,226,234,0.491304,1,1547.325609,69.670625,0.595132,184.026797,93.470777,2023-06-19 17:11:00,- (-)
193,42,SK Gaming,LEC,398,185,213,0.464824,1,1516.355727,73.493682,0.595138,187.580379,100.543805,2023-06-19 16:08:00,+1 (-)
331,518,Excel Esports,LEC,176,64,112,0.363636,-7,1465.320720,73.331276,0.595069,76.315569,40.952920,2023-06-19 16:08:00,-1 (-)


In [29]:
scoreboard_games.loc[scoreboard_games['League'] == 'LCS'].tail()

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision,League
2064,LCS/2023 Season/Summer Season,Immortals,Team Liquid,Team Liquid,Immortals,2023-06-16 21:08:00,0,1,2,21:09,21.150000,"Vi,Ahri,Kindred,Gwen,Wukong","Yuumi,Neeko,Sion,K'Sante,Syndra","Ornn,Sejuani,Vel'Koz,Aphelios,Milio","Jax,Lee Sin,Annie,Lucian,Nami","Solo (Colin Earnest),Kenvi,Bolulu,Tactical,Treatz","Summit,Pyosik,Haeri,Yeon (Sean Sung),CoreJJ",0.0,2.0,0.0,0.0,0.0,11.0,28821.0,47162.0,1.0,16.0,0.0,2.0,0.0,3.0,13.11,LCS/2023 Season/Summer Season_Week 1_11_1,LCS/2023 Season/Summer Season_Week 1_11,NaN,0,LCS
2065,LCS/2023 Season/Summer Season,FlyQuest,Golden Guardians,Golden Guardians,FlyQuest,2023-06-16 21:50:00,0,1,2,36:54,36.900000,"Vi,Milio,Lucian,Renekton,Poppy","Neeko,K'Sante,Yuumi,Wukong,Kindred","Gragas,Viego,Annie,Xayah,Rakan","Yone,Sejuani,Azir,Aphelios,Lulu","Impact,Spica,VicLa,Prince (Lee Chae-hwan),Vulc...","Licorice,River (Kim Dong-woo),Gori,Stixxay,huhi",3.0,2.0,1.0,2.0,4.0,11.0,68141.0,75894.0,20.0,19.0,1.0,1.0,0.0,2.0,13.11,LCS/2023 Season/Summer Season_Week 1_12_1,LCS/2023 Season/Summer Season_Week 1_12,NaN,0,LCS
2066,LCS/2023 Season/Summer Season,Evil Geniuses.NA,Dignitas,Dignitas,Evil Geniuses.NA,2023-06-16 22:52:00,0,1,2,35:42,35.700000,"Gwen,Vi,Maokai,Annie,LeBlanc","Neeko,Yuumi,Milio,Gragas,Kindred","Sion,Trundle,Ahri,Aphelios,Lulu","K'Sante,Sejuani,Cassiopeia,Xayah,Rakan","Revenge (Mohamed Kaddoura),Armao,jojopyun,UNF0...","Rich (Lee Jae-won),Santorin,Jensen,Tomo,Diamon...",2.0,3.0,0.0,2.0,4.0,8.0,58944.0,67150.0,10.0,18.0,1.0,1.0,0.0,1.0,13.11,LCS/2023 Season/Summer Season_Week 1_13_1,LCS/2023 Season/Summer Season_Week 1_13,NaN,0,LCS
2067,LCS/2023 Season/Summer Season,NRG,100 Thieves,NRG,100 Thieves,2023-06-16 23:52:00,1,0,1,34:34,34.566667,"Lucian,Ahri,Wukong,Malphite,Gwen","Neeko,Vi,Sejuani,Jax,None","K'Sante,Kha'Zix,Syndra,Aphelios,Milio","Renekton,Lee Sin,Annie,Zeri,Yuumi","Dhokla,Contractz,Palafox,FBI,IgNar","Ssumday,Closer (Can Çelik),Nukeduck,Doublelift...",3.0,2.0,2.0,0.0,11.0,2.0,70404.0,54517.0,18.0,3.0,1.0,1.0,3.0,0.0,13.11,LCS/2023 Season/Summer Season_Week 1_14_1,LCS/2023 Season/Summer Season_Week 1_14,NaN,0,LCS
2068,LCS/2023 Season/Summer Season,Cloud9,TSM,Cloud9,TSM,2023-06-17 00:49:00,1,0,1,29:25,29.416667,"Blitzcrank,Vi,Lucian,Rakan,Ezreal","Zeri,Neeko,Maokai,Gwen,Sylas","Gragas,Kindred,K'Sante,Aphelios,Milio","Malphite,Wukong,Ahri,Jhin,Karma","Fudge,Blaber,EMENES,Berserker (Kim Min-cheol),...","Hauntzer,Bugi,Insanity,WildTurtle,Chime",3.0,1.0,1.0,1.0,8.0,4.0,58072.0,50460.0,14.0,9.0,1.0,1.0,1.0,0.0,13.11,LCS/2023 Season/Summer Season_Week 1_15_1,LCS/2023 Season/Summer Season_Week 1_15,NaN,0,LCS


In [7]:
ratings.loc[(ratings["league"] == "LCS") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
12,86,Cloud9,LCS,1088,608,480,0.558824,3,1818.322567,58.694923,0.593644,452.294529,233.371000,2023-06-17 00:49:00,- (-)
37,163,Evil Geniuses.NA,LCS,369,194,175,0.525745,-1,1704.964575,49.623761,0.594273,168.467294,88.733942,2023-06-16 22:52:00,- (-)
44,470,Golden Guardians,LCS,290,112,178,0.386207,1,1691.529593,72.477893,0.595129,128.747835,68.571919,2023-06-16 21:50:00,- (-)
57,264,Team Liquid,LCS,863,499,364,0.578216,4,1660.655930,64.075842,0.594889,374.571308,199.794882,2023-06-16 21:08:00,- (-)
103,468,100 Thieves,LCS,447,205,242,0.458613,-2,1595.388516,61.357112,0.594880,182.997851,91.137430,2023-06-16 23:52:00,+1 (-)
109,414,FlyQuest,LCS,420,190,230,0.452381,-5,1586.654031,72.417693,0.595121,196.183430,103.356752,2023-06-16 21:50:00,- (-)
126,1,Counter Logic Gaming,LCS,696,336,360,0.482759,-5,1564.587089,67.193728,0.584382,321.127325,167.945502,2023-03-26 22:57:00,- (-)
163,4,TSM,LCS,1156,696,460,0.602076,-1,1538.990057,59.323507,0.594704,449.682904,236.701641,2023-06-17 00:49:00,+1 (-)
482,350,Immortals,LCS,340,168,172,0.494118,-1,1418.038500,64.042903,0.594845,148.165957,76.228266,2023-06-16 21:08:00,- (-)
554,351,NRG,LCS,72,23,49,0.319444,1,1363.803703,61.328370,0.594567,30.384963,15.043305,2023-06-16 23:52:00,- (-)


In [8]:
ratings.loc[
    (ratings["league"].isin(["LCK", "LPL", "LEC", "LCS"]))
    & (ratings["last_game_date"].dt.year == 2023)
]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
0,444,JD Gaming,LPL,742,440,302,0.592992,4,2047.693775,69.358026,0.540121,303.437995,155.588232,2023-06-18 12:13:00,- (-)
1,220,Gen.G,LCK,1024,614,410,0.599609,7,2025.678016,46.686849,0.565101,432.483068,229.308044,2023-06-22 08:52:00,- (+2)
2,69,T1,LCK,1446,960,486,0.663900,6,2006.859234,56.087865,0.566006,571.053414,301.503586,2023-06-21 12:34:00,- (-)
3,458,Bilibili Gaming,LPL,556,291,265,0.523381,2,1972.532740,67.104517,0.539238,251.695430,131.895096,2023-06-21 12:13:00,- (-)
4,219,KT Rolster,LCK,1057,628,429,0.594134,6,1959.257549,70.114352,0.570862,438.657278,232.655541,2023-06-22 11:36:00,- (+21)
6,211,G2 Esports,LEC,1016,580,436,0.570866,1,1863.036428,64.826826,0.594672,400.866629,210.259661,2023-06-19 19:18:00,+1 (-)
7,508,Dplus KIA,LCK,629,440,189,0.699523,-2,1844.971306,69.935458,0.566630,233.396783,122.167622,2023-06-22 11:36:00,-1 (-21)
9,205,Hanwha Life Esports,LCK,863,429,434,0.497103,4,1838.648796,69.823991,0.576615,366.674118,193.211938,2023-06-17 07:04:00,- (-)
12,86,Cloud9,LCS,1088,608,480,0.558824,3,1818.322567,58.694923,0.593644,452.294529,233.371000,2023-06-17 00:49:00,- (-)
13,244,LNG Esports,LPL,842,423,419,0.502375,-2,1815.361153,67.491745,0.552233,387.907519,206.445421,2023-06-21 12:13:00,- (-)


In [9]:
leagues = ["LCK", "LPL", "LEC", "LCS"]
for league in leagues:
    print(league)
    lst = ratings.loc[
        (ratings["league"] == league) & (ratings["last_game_date"].dt.year == 2023),
        "team",
    ].unique()
    print(", ".join(map(lambda x: f"'{x}'", lst)))

LCK
'Gen.G', 'T1', 'KT Rolster', 'Dplus KIA', 'Hanwha Life Esports', 'Liiv SANDBOX', 'Kwangdong Freecs', 'DRX', 'OKSavingsBank BRION', 'Nongshim RedForce'
LPL
'JD Gaming', 'Bilibili Gaming', 'LNG Esports', 'Top Esports', 'Oh My God', 'Weibo Gaming', 'Royal Never Give Up', 'EDward Gaming', 'Team WE', 'Ninjas in Pyjamas.CN', 'Rare Atom', 'FunPlus Phoenix', 'ThunderTalk Gaming', 'Ultra Prime', 'LGD Gaming', 'Invictus Gaming', 'Anyone's Legend'
LEC
'G2 Esports', 'MAD Lions', 'Team BDS', 'Fnatic', 'Team Vitality', 'KOI (Spanish Team)', 'Team Heretics', 'Astralis', 'SK Gaming', 'Excel Esports'
LCS
'Cloud9', 'Evil Geniuses.NA', 'Golden Guardians', 'Team Liquid', '100 Thieves', 'FlyQuest', 'Counter Logic Gaming', 'TSM', 'Immortals', 'NRG', 'Dignitas'


In [10]:
leagues = {
    "LCK": "LoL Champions Korea",
    "LPL": "Tencent LoL Pro League",
    "LEC": "LoL EMEA Championship",
    "LCS": "League of Legends Championship Series",
    "MSI": "Mid-Season Invitational",
    "WCS": "World Championship",
}

tournaments = get_tournaments(where=f'T.Year=2023 and T.League="{leagues["LCK"]}"')
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,LCK 2023 Spring,LCK/2023 Season/Spring Season,2023-01-18,2023-03-19,LoL Champions Korea,Korea,None,LCK 2023 Spring,Spring,1,Primary,0,0,1,2023,1,1
1,LCK 2023 Spring Playoffs,LCK/2023 Season/Spring Playoffs,2023-03-22,2023-04-09,LoL Champions Korea,Korea,Offline,LCK 2023 Spring Playoffs,Spring,1,Primary,0,1,1,2023,1,1
2,LCK 2023 Summer,LCK/2023 Season/Summer Season,2023-06-07,2023-08-06,LoL Champions Korea,Korea,None,LCK 2023 Summer,Summer,2,Primary,0,0,1,2023,1,1
3,LCK 2023 Summer Playoffs,LCK/2023 Season/Summer Playoffs,2023-08-08,2023-08-20,LoL Champions Korea,Korea,Offline,LCK 2023 Summer Playoffs,Summer,2,Primary,0,1,1,2023,1,1


In [11]:
page = "LCK/2023 Season/Summer Season"
match_schedules = get_match_schedule(where=f'MS.OverviewPage="{page}"').sort_values(
    by=["DateTime UTC"], ignore_index=True
)


teams = {}
lst_team = []
for team_name in match_schedules[["Team1", "Team2"]].unstack().unique():
    team_id = get_team_id(teams_id, team_name)
    row = ratings.loc[ratings["team"] == team_name].iloc[0]
    teams[team_id] = Team(
        name=team_name,
        league=row["league"],
        team_id=row["team_id"],
        win=row["win"],
        loss=row["loss"],
        streak=row["streak"],
        point=row["point"],
        rd=row["rd"],
        sigma=row["sigma"],
        last_game_date=row["last_game_date"],
    )
    lst_team.append(teams[team_id])

In [12]:
match_schedules = match_schedules.loc[match_schedules["Winner"].isna()].reset_index(
    drop=True
)
match_schedules.head()

,Team1,Team2,Team1Final,Team2Final,Winner,Team1Points,Team2Points,Team1PointsTB,Team2PointsTB,Team1Score,Team2Score,Team1Poster,Team2Poster,Team1Advantage,Team2Advantage,FF,IsNullified,Player1,Player2,MatchDay,DateTime UTC,HasTime,DST,IsFlexibleStart,IsReschedulable,OverrideAllowPredictions,OverrideDisallowPredictions,IsTiebreaker,OverviewPage,ShownName,ShownRound,BestOf,Round,Phase,N MatchInPage,Tab,N MatchInTab,N TabInPage,N Page,Patch,PatchPage,Hotfix,DisabledChampions,PatchFootnote,InitialN MatchInTab,InitialPageAndTab,GroupName,Stream,StreamDisplay,Venue,CastersPBP,CastersColor,Casters,MVP,MVPPoints,VodInterview,VodHighlights,InterviewWith,Recap,Reddit,QQ,Wanplus,WanplusId,PageAndTeam1,PageAndTeam2,Team1Footnote,Team2Footnote,Footnote,UniqueMatch,MatchId,DateTime UTC__precision
0,Kwangdong Freecs,Nongshim RedForce,Kwangdong Freecs,Nongshim RedForce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3,2023-06-23 08:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,25,Week 3,5,3,1,13.11,Patch 13.11,NaN,NaN,NaN,5,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Kwangdong Freecs,LCK/2023 Season/Summer Season_Nongshim RedForce,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_5,LCK/2023 Season/Summer Season_Week 3_5,0
1,Hanwha Life Esports,Liiv SANDBOX,Hanwha Life Esports,Liiv SANDBOX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3,2023-06-23 10:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,26,Week 3,6,3,1,13.11,Patch 13.11,NaN,NaN,NaN,6,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Hanwha Life Esports,LCK/2023 Season/Summer Season_Liiv SANDBOX,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_6,LCK/2023 Season/Summer Season_Week 3_6,0
2,Dplus KIA,Gen.G,Dplus KIA,Gen.G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,4,2023-06-24 06:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,27,Week 3,7,3,1,13.11,Patch 13.11,NaN,NaN,NaN,7,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Dplus KIA,LCK/2023 Season/Summer Season_Gen.G,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_7,LCK/2023 Season/Summer Season_Week 3_7,0
3,T1,KT Rolster,T1,KT Rolster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,4,2023-06-24 08:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,28,Week 3,8,3,1,13.11,Patch 13.11,NaN,NaN,NaN,8,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_T1,LCK/2023 Season/Summer Season_KT Rolster,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_8,LCK/2023 Season/Summer Season_Week 3_8,0
4,Nongshim RedForce,DRX,Nongshim RedForce,DRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,5,2023-06-25 06:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,29,Week 3,9,3,1,13.11,Patch 13.11,NaN,NaN,NaN,9,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Nongshim RedForce,LCK/2023 Season/Summer Season_DRX,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_9,LCK/2023 Season/Summer Season_Week 3_9,0


In [13]:
glicko = GlickoSystem()

In [14]:
matches = match_schedules.head(10)
matches.head()

,Team1,Team2,Team1Final,Team2Final,Winner,Team1Points,Team2Points,Team1PointsTB,Team2PointsTB,Team1Score,Team2Score,Team1Poster,Team2Poster,Team1Advantage,Team2Advantage,FF,IsNullified,Player1,Player2,MatchDay,DateTime UTC,HasTime,DST,IsFlexibleStart,IsReschedulable,OverrideAllowPredictions,OverrideDisallowPredictions,IsTiebreaker,OverviewPage,ShownName,ShownRound,BestOf,Round,Phase,N MatchInPage,Tab,N MatchInTab,N TabInPage,N Page,Patch,PatchPage,Hotfix,DisabledChampions,PatchFootnote,InitialN MatchInTab,InitialPageAndTab,GroupName,Stream,StreamDisplay,Venue,CastersPBP,CastersColor,Casters,MVP,MVPPoints,VodInterview,VodHighlights,InterviewWith,Recap,Reddit,QQ,Wanplus,WanplusId,PageAndTeam1,PageAndTeam2,Team1Footnote,Team2Footnote,Footnote,UniqueMatch,MatchId,DateTime UTC__precision
0,Kwangdong Freecs,Nongshim RedForce,Kwangdong Freecs,Nongshim RedForce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3,2023-06-23 08:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,25,Week 3,5,3,1,13.11,Patch 13.11,NaN,NaN,NaN,5,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Kwangdong Freecs,LCK/2023 Season/Summer Season_Nongshim RedForce,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_5,LCK/2023 Season/Summer Season_Week 3_5,0
1,Hanwha Life Esports,Liiv SANDBOX,Hanwha Life Esports,Liiv SANDBOX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3,2023-06-23 10:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,26,Week 3,6,3,1,13.11,Patch 13.11,NaN,NaN,NaN,6,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Hanwha Life Esports,LCK/2023 Season/Summer Season_Liiv SANDBOX,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_6,LCK/2023 Season/Summer Season_Week 3_6,0
2,Dplus KIA,Gen.G,Dplus KIA,Gen.G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,4,2023-06-24 06:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,27,Week 3,7,3,1,13.11,Patch 13.11,NaN,NaN,NaN,7,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Dplus KIA,LCK/2023 Season/Summer Season_Gen.G,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_7,LCK/2023 Season/Summer Season_Week 3_7,0
3,T1,KT Rolster,T1,KT Rolster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,4,2023-06-24 08:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,28,Week 3,8,3,1,13.11,Patch 13.11,NaN,NaN,NaN,8,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_T1,LCK/2023 Season/Summer Season_KT Rolster,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_8,LCK/2023 Season/Summer Season_Week 3_8,0
4,Nongshim RedForce,DRX,Nongshim RedForce,DRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,5,2023-06-25 06:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,29,Week 3,9,3,1,13.11,Patch 13.11,NaN,NaN,NaN,9,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Nongshim RedForce,LCK/2023 Season/Summer Season_DRX,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 3_9,LCK/2023 Season/Summer Season_Week 3_9,0


In [15]:
columns = ["Team1", "Point1", "Winprob1", "Winprob2", "Point2", "Team2"]
probs = pd.DataFrame(columns=columns)
for row in matches.itertuples():
    team1_name = row.Team1
    team2_name = row.Team2
    team1 = teams[get_team_id(teams_id, team1_name)]
    team2 = teams[get_team_id(teams_id, team2_name)]
    lst = [
        team1.name,
        team1.point,
        team1.get_win_probability(glicko, team2) * 100,
        team2.get_win_probability(glicko, team1) * 100,
        team2.point,
        team2.name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs

,Team1,Point1,Winprob1,Winprob2,Point2,Team2
0,Kwangdong Freecs,1706.823392,68.083336,31.911083,1573.088248,Nongshim RedForce
1,Hanwha Life Esports,1838.648796,64.645018,35.359843,1731.309680,Liiv SANDBOX
2,Dplus KIA,1844.971306,26.327856,73.409864,2025.678016,Gen.G
3,T1,2006.859234,56.647296,43.296195,1959.257549,KT Rolster
4,Nongshim RedForce,1573.088248,42.133190,57.827121,1628.813367,DRX
5,OKSavingsBank BRION,1603.193365,21.027170,78.984019,1838.648796,Hanwha Life Esports
6,Hanwha Life Esports,1838.648796,49.111828,50.888239,1844.971306,Dplus KIA
7,KT Rolster,1959.257549,80.698142,19.490715,1706.823392,Kwangdong Freecs
8,Gen.G,2025.678016,91.494960,8.273513,1603.193365,OKSavingsBank BRION
9,DRX,1628.813367,35.979708,64.190253,1731.309680,Liiv SANDBOX


In [16]:
matches_bo3 = matches.loc[matches["BestOf"] == "3"]
columns = ["Team1", "Point1", "2:0", "2:1", "1:2", "0:2", "Point2", "Team2"]
probs = pd.DataFrame(columns=columns)
for row in matches_bo3.itertuples():
    team1_name = row.Team1
    team2_name = row.Team2
    team1 = teams[get_team_id(teams_id, team1_name)]
    team2 = teams[get_team_id(teams_id, team2_name)]
    team1_prob = team1.get_win_probability(glicko, team2)
    team2_prob = team2.get_win_probability(glicko, team1)
    lst = [
        team1.name,
        team1.point,
        team1_prob**2 * 100,
        (team1_prob**2) * team2_prob * 2 * 100,
        (team2_prob**2) * team1_prob * 2 * 100,
        team2_prob**2 * 100,
        team2.point,
        team2_name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs["Winprob1"] = probs[["2:0", "2:1"]].sum(axis=1)
probs["Winprob2"] = probs[["1:2", "0:2"]].sum(axis=1)
columns = [
    "Team1",
    "Point1",
    "Winprob1",
    "2:0",
    "2:1",
    "1:2",
    "0:2",
    "Winprob2",
    "Point2",
    "Team2",
]
probs[columns]

,Team1,Point1,Winprob1,2:0,2:1,1:2,0:2,Winprob2,Point2,Team2
0,Kwangdong Freecs,1706.823392,75.937154,46.353406,29.583748,13.866087,10.183172,24.049259,1573.088248,Nongshim RedForce
1,Hanwha Life Esports,1838.648796,71.343387,41.789783,29.553604,16.165373,12.503185,28.668558,1731.309680,Liiv SANDBOX
2,Dplus KIA,1844.971306,17.108458,6.931560,10.176898,28.376206,53.890081,82.266287,2025.678016,Gen.G
3,T1,2006.859234,59.875932,32.089161,27.786772,21.237757,18.745605,39.983362,1959.257549,KT Rolster
4,Nongshim RedForce,1573.088248,38.283064,17.752057,20.531007,28.178475,33.439760,61.618234,1628.813367,DRX
5,OKSavingsBank BRION,1603.193365,11.405847,4.421419,6.984428,26.235496,62.384753,88.620248,1838.648796,Hanwha Life Esports
6,Hanwha Life Esports,1838.648796,48.667914,24.119716,24.548198,25.436124,25.896128,51.332252,1844.971306,Dplus KIA
7,KT Rolster,1959.257549,90.507349,65.121901,25.385448,6.131250,3.798880,9.930130,1706.823392,Kwangdong Freecs
8,Gen.G,2025.678016,97.565334,83.713277,13.852058,1.252585,0.684510,1.937095,1603.193365,OKSavingsBank BRION
9,DRX,1628.813367,29.564756,12.945394,16.619362,29.650076,41.203885,70.853961,1731.309680,Liiv SANDBOX


In [17]:
matches_bo5 = matches.loc[matches["BestOf"] == "5"]
columns = [
    "Team1",
    "Point1",
    "Winprob1",
    "3:0",
    "3:1",
    "3:2",
    "2:3",
    "1:3",
    "0:3",
    "Winprob2",
    "Point2",
    "Team2",
]
probs = pd.DataFrame(columns=columns)
for row in matches_bo5.itertuples():
    team1_name = row.Team1
    team2_name = row.Team2
    team1 = teams[get_team_id(teams_id, team1_name)]
    team2 = teams[get_team_id(teams_id, team2_name)]
    team1_prob = team1.get_win_probability(glicko, team2)
    team2_prob = team2.get_win_probability(glicko, team1)
    lst = [
        team1.name,
        team1.point,
        0,
        team1_prob**3 * 100,
        (team1_prob**3) * team2_prob * 3 * 100,
        (team1_prob**3) * (team2_prob**2) * 6 * 100,
        (team2_prob**3) * (team1_prob**2) * 6 * 100,
        (team2_prob**3) * team1_prob * 3 * 100,
        team2_prob**3 * 100,
        0,
        team2.point,
        team2.name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs["Winprob1"] = probs[["3:0", "3:1", "3:2"]].sum(axis=1)
probs["Winprob2"] = probs[["2:3", "1:3", "0:3"]].sum(axis=1)
probs

,Team1,Point1,Winprob1,3:0,3:1,3:2,2:3,1:3,0:3,Winprob2,Point2,Team2
